In [ ]:
import sys
import os
import numpy as np
import numpy.random as nrand
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import copy
import multiprocessing
from sklearn.linear_model import LinearRegression
import scipy
from scipy.linalg import hadamard
import itertools
import time
from scipy.sparse import csr_matrix
currentdir = os.path.abspath('')
targetdir = os.path.dirname(currentdir)+'/utils'
sys.path.append(targetdir)
from utils import *

In [1]:
def worker(start):
    """
    Worker function used for multithreading.
    start: the index of the global FL list showing where to start analysis.
    """
    plot_dict = {f'{metric}_T':[],f'{metric}_E':[],'std':[]}
    error_list = [0.02,0.04,0.06,0.08] # introduce measurement error of different level
    for i in range(start+1,start+11):
        #print(i)
        for j in range(10):
            # Calculate ruggedness of the True FL
            ruggedness_T = get_ruggedness(NK_landscape_list[i][j])
            for k in error_list:
                # Introduce measurement error
                NK_landscape_error = Add_Error(NK_landscape_list[i][j],k) 
                # Calculate ruggedness of Observed FL (with measurement error)
                ruggedness_E = get_ruggedness(NK_landscape_error)
                # Record std, ruggedness of Ture and Observed FL for plotting
                plot_dict[f'{metric}_T'].append(ruggedness_T)
                plot_dict[f'{metric}_E'].append(ruggedness_E)
                plot_dict['std'].append(k)
    plot_df = pd.DataFrame(plot_dict)
    return plot_df

In [ ]:
model_list = ['NK','RMF','Polynomial'] # Type of theoretical landscape model: 'NK', 'RMF', 'Polynomial'
N_list = [5] # Number of variable sites: 5, 10, 15
metric_list = ['N_max', 'epi', 'r_s', 'open_ratio','E', 'gamma', 'adptwalk_steps', 'adptwalk_probs']
    # Type of metrics: 'N_max', 'epi', 'r_s', 'open_ratio','E', 'gamma', 'adptwalk_steps', 'adptwalk_probs'

for N in N_list:
    print(f"N:{N}")
    load_pregenerated_data(N)
    
    for model in model_list:
        print(f"  model:{model}")
        # Load pre-generated Fitness landscapes
        with open(f'../FL_data_100X10/{model}_{N}_landscape_list_100X10.pkl', 'rb') as f:
            NK_landscape_list = pickle.load(f)

        for metric in metric_list:
            print(f"    {metric}")
            # Get the ruggendess function for calculating specific ruggedness measure
            get_ruggedness = get_ruggedness_function(metric,N,gt_code=NK_landscape_list[1][0][:,:N])
            # Multithreading
            jobs = []
            with multiprocessing.Pool(10) as p:
                res_list = p.map(worker, [i*10 for i in range(10)])
            # Gather data
            plot_df = pd.concat(res_list)
            
            # Uncomment only if you want to overwrite plot data files in plot_data folder.
            # with open(f'./plot_data/plot_df_{model}_{N}_{metric}.pkl','wb') as f:
            #     pickle.dump(plot_df,f)